In [42]:
from gensim import corpora, models, similarities
import gensim

In [43]:
import json 
import gzip 
#from StringIO import StringIO
import urllib.request
import requests, zipfile, io

def parse(url): 
    
    r = requests.get(url)
    g = gzip.open(io.BytesIO(r.content))
    for l in g:
        yield eval(l)

import json 
import gzip 
def parse(url): 
    g = gzip.open(url, 'r') 
    for l in g: 
        yield eval(l)

In [44]:
import gensim
def myReviews(review_files_urls, catFilter = None, asin = None, concatenate = False):
    preprocess_reviewText = []
    for url in review_files_urls:
        print("Processing url: " + url)
        review_item_iterator = iter(parse(url))
        it = 0
        conc_count = 0
        for review_item in review_item_iterator:
            it += 1
        #for it in range(1000000):
            if it%100000==0:
                print("it="+str(it))

            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)


            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
                if concatenate==False:
                    yield preprocess_reviewText_i
                else:
                    conc_count += 1
                    preprocess_reviewText.extend(preprocess_reviewText_i)
                    print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

In [45]:
base_url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/"
from bs4 import BeautifulSoup
import urllib.request
import re

html = urllib.request.urlopen(base_url)
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())
all_html_links = soup.find_all('a')


for links in all_html_links:
    if re.match("^review.*[^_5|_10].json.gz$", links["href"]):
        print(links["href"])

In [46]:
review_file_names = [links["href"] for links in all_html_links if re.match("^review.*[^_5|_10].json.gz$", links["href"])]
#print(review_file_names)
review_file_urls = [base_url + review_file_name for review_file_name in review_file_names]
#print(review_file_urls)

In [47]:

    
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
if os.path.isfile(dictionaryFileName)==False:
    #dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    dictionary = corpora.Dictionary(myReviews(review_file_urls))
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_All_Beauty.json.gz
Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_All_Credit_Cards.json.gz
Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_All_Electronics.json.gz
Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Alternative_Rock.json.gz
Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Coins.json.gz
Processing url: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Fashion.json.gz


KeyboardInterrupt: 

In [ ]:
def myCorpus(review_file_urls, catFilter = None, asin = None, concatenate = False):
    for review in myReviews(review_file_urls,catFilter,asin,concatenate):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [ ]:
import os
tfidfFileName ='../../download/word2vec/reviews_all_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    #tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf = models.TfidfModel(myCorpus(review_file_urls))
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

In [ ]:
import pickle

tfidf_apple_5s_FileName ='../../download/word2vec/apple_5s_review.tfidf'
if os.path.isfile(tfidf_apple_5s_FileName)==False:
    apple_5s_corpus = next(myCorpus(asin="B00F3J4B5S",concatenate=True))
    apple_5s_corpus_tfidf = tfidf[apple_5s_corpus]  
    pickle.dump(apple_5s_corpus_tfidf, open(tfidf_apple_5s_FileName, 'wb'))
else:
    apple_5s_corpus_tfidf = pickle.load( open( tfidf_apple_5s_FileName, "rb" ) )
    
    


In [ ]:
print([(dictionary.get(tokenid),tfidf_value) for (tokenid,tfidf_value) in apple_5s_corpus_tfidf if tfidf_value >0.1])

In [ ]:
print(tfidf)